## Reduction & Fusion UERC

--> V2.0

Train: UERC train

Test: UERC test


## Dimensionality Reduction

In [4]:
import numpy as np
import pandas as pd
import csv

from sklearn.preprocessing import MinMaxScaler, StandardScaler
from sklearn.decomposition import PCA
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis as LDA
    
def pca_reduce(csvfileTrain, csvfileTest, components):
    dataset = pd.read_csv(csvfileTrain)
    X = dataset.drop('class', axis=1)  
    y = dataset['class']
    #sc_X = MinMaxScaler(feature_range=[0, 1])
    sc_X = StandardScaler()
    X = sc_X.fit_transform(X)
    pca = PCA(n_components = components)
    X_red = pca.fit_transform(X)
    
    head = []
    head.append('class')
    for i in range(1, len(X_red[0])+1):
        head.append('v'+str(i))

    head_np = []
    head_np.append(head)
    newcsvfile = csvfileTrain[:-4] + "_redPCA_train_" + str(len(X_red[0])) + ".csv"
    with open(newcsvfile, 'w') as csvFile:
        writer = csv.writer(csvFile)
        writer.writerows(head_np)
        for i in range(len(X)):
            writer.writerow(np.concatenate((np.array([y[i]]), X_red[i]), axis=0) )
        csvFile.close()
    print("New file: {}".format(newcsvfile))
    
    dataset = pd.read_csv(csvfileTest)
    X = dataset.drop('class', axis=1)  
    y = dataset['class']
    sc_X = StandardScaler()
    X = sc_X.fit_transform(X)
    #pca = PCA(n_components = components)
    X_red_test = pca.transform(X)
    
    head = []
    head.append('class')
    for i in range(1, len(X_red_test[0])+1):
        head.append('v'+str(i))

    head_np = []
    head_np.append(head)
    newcsvfile = csvfileTest[:-4] + "_redPCA_test_" + str(len(X_red_test[0])) + ".csv"
    with open(newcsvfile, 'w') as csvFile:
        writer = csv.writer(csvFile)
        writer.writerows(head_np)
        for i in range(len(X)):
            writer.writerow(np.concatenate((np.array([y[i]]), X_red_test[i]), axis=0) )
        csvFile.close()
    print("New file: {}".format(newcsvfile))
    return
    
def lda_reduce(csvfileTrain, csvfileTest, components):
    dataset = pd.read_csv(csvfileTrain)
    X = dataset.drop('class', axis=1)  
    y = dataset['class']
    sc_X = StandardScaler()
    X = sc_X.fit_transform(X)
    lda = LDA(n_components = components)
    X_red = lda.fit_transform(X,y)
    
    head = []
    head.append('class')
    for i in range(1, len(X_red[0])+1):
        head.append('v'+str(i))

    head_np = []
    head_np.append(head)
    newcsvfile = csvfileTrain[:-4] + "_redLDA_train_" + str(len(X_red[0])) + ".csv"
    with open(newcsvfile, 'w') as csvFile:
        writer = csv.writer(csvFile)
        writer.writerows(head_np)
        for i in range(len(X)):
            writer.writerow(np.concatenate((np.array([y[i]]), X_red[i]), axis=0) )
        csvFile.close()
    print("New file: {}".format(newcsvfile))
    
    dataset = pd.read_csv(csvfileTest)
    X = dataset.drop('class', axis=1)  
    y = dataset['class']
    sc_X = StandardScaler()
    X = sc_X.fit_transform(X)
    #lda = LDA(n_components = components)
    X_red_test = lda.transform(X)
    
    head = []
    head.append('class')
    for i in range(1, len(X_red_test[0])+1):
        head.append('v'+str(i))

    head_np = []
    head_np.append(head)
    newcsvfile = csvfileTest[:-4] + "_redLDA_test_" + str(len(X_red_test[0])) + ".csv"
    with open(newcsvfile, 'w') as csvFile:
        writer = csv.writer(csvFile)
        writer.writerows(head_np)
        for i in range(len(X_red_test)):
            writer.writerow(np.concatenate((np.array([y[i]]), X_red_test[i]), axis=0) )
        csvFile.close()
    print("New file: {}".format(newcsvfile))
    return

## Fusion

In [5]:
import pandas as pd
import csv
import numpy as np

# Input:
    # csvFiles: list of csv files -> "class", "v1", "v2"... "vn"
    # fusionType: 1 -> concatenated (number of rows must be the same)
    #             2 -> suma (number of columns must be the same)
    #             3 -> max (number of columns must be the same)
# Output:
    # csvFusedFile: Final output file name
def fusionFeatureVectors(csvFiles=[], fusionType = 1, outputFileName="output_fused_file.csv"):
    if (fusionType == 1):
        if len(csvFiles) >= 2:
            data = pd.read_csv(csvFiles[0])
            dataFinal_X = data.drop('class', axis=1)
            dataFinal_X = dataFinal_X.values
            dataFinal_y = data['class'].values
            print("{} - rows: {} -  columns: {}".format(csvFiles[0], str(len(dataFinal_X)), str(len(dataFinal_X[0]))))
                
            for i in range(1, len(csvFiles)):                
                data = pd.read_csv(csvFiles[i])
                X = data.drop('class', axis=1)
                X = X.values
                y = data['class'].values
                print("{} - rows: {} -  columns: {}".format(csvFiles[i], str(len(X)), str(len(X[0]))))
                # number of rows should be the same
                if len(dataFinal_y) == len(y) and np.all(dataFinal_y == y ):
                    dataFinal_X = np.concatenate((dataFinal_X, X),axis=1)
                else:
                    print("Labels do not match! - {}".format(csvFiles[i]))
                    return
                        
    if (fusionType == 2 or fusionType == 3):
        if len(csvFiles) >= 2:
            data = pd.read_csv(csvFiles[0])
            dataFinal_X = data.drop('class', axis=1)
            dataFinal_X = dataFinal_X.values
            dataFinal_y = data['class'].values
            print("{} - rows: {} -  columns: {}".format(csvFiles[0], str(len(dataFinal_X)), str(len(dataFinal_X[0]))))
            
            for i in range(1, len(csvFiles)):
                data = pd.read_csv(csvFiles[i])
                X = data.drop('class', axis=1)
                X = X.values
                y = data['class'].values
                print("{} - rows: {} -  columns: {}".format(csvFiles[i], str(len(X)), str(len(X[0]))))
                # number of rows columns should be the same
                if len(dataFinal_X[0]) == len(X[0]) and len(dataFinal_y) == len(y) and np.all(dataFinal_y == y):
                    for j in range(len(dataFinal_X)):
                        for l in range(len(dataFinal_X[0])):
                            if fusionType == 2:
                                dataFinal_X[j][l] =  dataFinal_X[j][l] + X[j][l]
                            if fusionType == 3:
                                dataFinal_X[j][l] =  np.maximu(dataFinal_X[j][l], X[j][l])
                else:
                    print("Labels or Rows do not match! - {}".format(csvFiles[i]))
                    return
    
    head = []
    head.append('class')
    for i in range(1, len(dataFinal_X[0])+1):
        head.append('v'+str(i))

    head_np = []
    head_np.append(head)
    #newcsvfile = outputFileName + ".csv"
    print("{} - rows: {} -  columns: {}".format(outputFileName, str(len(dataFinal_X)), str(len(dataFinal_X[0]))))
    with open(outputFileName, 'w') as csvFile:
        writer = csv.writer(csvFile)
        writer.writerows(head_np)
        for k in range(len(dataFinal_X)):
            writer.writerow(np.concatenate((np.array([dataFinal_y[k]]), dataFinal_X[k]), axis=0) )
        csvFile.close()
    #print("CSV File created: {}".format(newcsvfile))    
    

## Get Metrics

In [6]:
import pandas as pd

from scipy import spatial

def chiSquaredDistance(a,b):
    res = np.sqrt(0.5*np.sum((a-b)**2/(a+b+1e-6)))
    return res

def chiSquared2(a,b):
    #res = np.sqrt(0.5*np.sum((a-b)**2/(a+b+1e-6)))
    return 0.5*np.sum((a-b)**2/(a+b+1e-6))

def cosineDistance(a,b):
    return spatial.distance.cosine(a,b)

def cosineDistance2(a,b):
    return 1 - spatial.distance.cosine(a,b) 

def euclideanDistance(a,b):
    return spatial.distance.euclidean(a,b)

# Input: csvfile: data file with feature vectors and labels -> 'class', 'v1', 'v2'... 'vn'
#        rank: desire rank (eg. rank-1, rank-5, rank-10)
#        distance_technique: 1 -> chisquared, 2 -> cosine
# Output: rankbins
def getMetrics(csvfilename, rank=1, distance_techinque=1):
    data = pd.read_csv(csvfilename)
    #Xdf = data.drop(['filename','class'], axis=1)  
    Xdf = data.drop('class', axis=1)  
    ydf = data['class']
    X = Xdf.values
    y = ydf.values
    s = (len(X), len(X), 2)
    y_pred = np.zeros(s)
    print("rows: {} -  columns: {}".format(len(X), len(X[0])))
                                                             
    # Normalization
    
    # Get distances matrix
    if (distance_techinque == 1):
        for i in range(len(X)):        
            for j in range (len(X)):
                dis = chiSquaredDistance(X[i], X[j])
                y_pred[i][j] = [dis, y[j]]
    if (distance_techinque == 2):
        for i in range(len(X)):        
            for j in range (len(X)):                   
                dis = cosineDistance(X[i], X[j]) 
                y_pred[i][j] = [dis, y[j]]
    if (distance_techinque == 3):
        for i in range(len(X)):        
            for j in range (len(X)):                                   
                dis = euclideanDistance(X[i], X[j])                         
                y_pred[i][j] = [dis, y[j]] 
            
    print("Distances calculated ...")
          
    # Order matrix acording to distance
    for i in range(len(y_pred)):
        b = y_pred[i]
        y_pred[i] = b[b[:,0].argsort()] 
    
    
    # Get Rank counting
    rankbins = np.zeros(rank+2)
    for i in range (len(y_pred)):        
        for j in range (len(y_pred)):
            if y_pred[i][j][0] == 0.0: #if distance is 0, it's the same vector                
                continue
            else:
                if j > rank: #we are interested in evaluate only the rank-n                    
                    rankbins[rank+1] += 1
                    break;
                else: 
                    if y[i] == y_pred[i][j][1]:                                                 
                        rankbins[j] += 1                        
                        break 
    
    rank_rate = rankbins/sum(rankbins)
    #print(rank_rate[0], rank_rate[1], rank_rate[2], np.sum(rank_rate[:1]), np.sum(rank_rate[:2]))
    rank_vector = []
    for r in range(2,rank+2):
        rank_vector.append(np.sum(rank_rate[:r]))
    return rank_vector

## Test UERC

### Rank Evaluation Handcrafted 

In [8]:
res = getMetrics(
    csvfilename = "vgg16_calculated_features/vgg16_1stage_uerc180class.csv", 
    rank = 1,
    distance_techinque=1) #chisquared distance
res

rows: 1800 -  columns: 4096
Distances calculated ...


[0.092777777777777778]

In [7]:
res = getMetrics(
    csvfilename = "vgg16_calculated_features/vgg16_1stage_uerc180class.csv", 
    rank = 1,
    distance_techinque=2) #cosine distance
res

rows: 1800 -  columns: 4096
Distances calculated ...


[0.105]

In [11]:
res = getMetrics(
    csvfilename = "awetoolbox_calculated_features/_output_uerctest_180class/uerctest-awet_lpq.csv", 
    rank = 1,
    distance_techinque=1) #chisquared distance
res

rows: 1800 -  columns: 9216
Distances calculated ...


[0.18111111111111111]

## TEST UERC

## Chi squared distance

In [12]:
from glob import glob 

ufiles = glob("awetoolbox_calculated_features/_output_uerctest_180class/*.csv")
files = sorted(ufiles)

for i in range(len(files)):
    print("--> {}".format(files[i]))
    res = getMetrics(
        csvfilename = files[i], 
        rank = 15,
        distance_techinque=1) #chisquared distance
    print(res)


--> awetoolbox_calculated_features/_output_uerctest_180class/uerctest-awet_bsif.csv
rows: 1800 -  columns: 9216
Distances calculated ...
[0.21111111111111111, 0.26611111111111113, 0.30722222222222223, 0.3338888888888889, 0.36333333333333334, 0.38333333333333336, 0.39944444444444444, 0.41277777777777774, 0.42944444444444441, 0.44499999999999995, 0.45999999999999996, 0.4705555555555555, 0.48333333333333328, 0.49166666666666664, 0.50055555555555553]
--> awetoolbox_calculated_features/_output_uerctest_180class/uerctest-awet_dsift.csv
rows: 1800 -  columns: 12800
Distances calculated ...
[0.14999999999999999, 0.2038888888888889, 0.24166666666666667, 0.27277777777777779, 0.2927777777777778, 0.31888888888888889, 0.33277777777777778, 0.35166666666666668, 0.3666666666666667, 0.38222222222222224, 0.39555555555555555, 0.40999999999999998, 0.42166666666666663, 0.43388888888888888, 0.44388888888888889]
--> awetoolbox_calculated_features/_output_uerctest_180class/uerctest-awet_gabor.csv
rows: 1800 -

## Cosine distance

In [13]:
from glob import glob 

ufiles = glob("awetoolbox_calculated_features/_output_uerctest_180class/*.csv")
files = sorted(ufiles)

for i in range(len(files)):
    print("--> {}".format(files[i]))
    res = getMetrics(
        csvfilename = files[i], 
        rank = 15,
        distance_techinque=2) #cosine distance
    print(res)


--> awetoolbox_calculated_features/_output_uerctest_180class/uerctest-awet_bsif.csv
rows: 1800 -  columns: 9216
Distances calculated ...
[0.15333333333333332, 0.20944444444444443, 0.25166666666666665, 0.27999999999999997, 0.30555555555555552, 0.33166666666666667, 0.35277777777777775, 0.37, 0.38666666666666666, 0.39944444444444444, 0.40777777777777779, 0.41833333333333333, 0.42999999999999999, 0.43888888888888888, 0.45111111111111107]
--> awetoolbox_calculated_features/_output_uerctest_180class/uerctest-awet_dsift.csv
rows: 1800 -  columns: 12800
Distances calculated ...
[0.15444444444444444, 0.20722222222222222, 0.24555555555555555, 0.27500000000000002, 0.29833333333333334, 0.31888888888888889, 0.33611111111111114, 0.35500000000000004, 0.3716666666666667, 0.38444444444444448, 0.39333333333333337, 0.41166666666666668, 0.42499999999999999, 0.43444444444444441, 0.44611111111111112]
--> awetoolbox_calculated_features/_output_uerctest_180class/uerctest-awet_gabor.csv
rows: 1800 -  columns: 

## Euclidean distance

In [14]:
from glob import glob 

ufiles = glob("awetoolbox_calculated_features/_output_uerctest_180class/*.csv")
files = sorted(ufiles)

for i in range(len(files)):
    print("--> {}".format(files[i]))
    res = getMetrics(
        csvfilename = files[i], 
        rank = 15,
        distance_techinque=3) #euclidean distance
    print(res)


--> awetoolbox_calculated_features/_output_uerctest_180class/uerctest-awet_bsif.csv
rows: 1800 -  columns: 9216
Distances calculated ...
[0.12277777777777778, 0.16555555555555557, 0.20055555555555557, 0.2277777777777778, 0.24500000000000002, 0.26333333333333336, 0.27833333333333332, 0.29222222222222222, 0.30777777777777776, 0.31944444444444442, 0.32888888888888884, 0.34277777777777774, 0.35222222222222216, 0.36333333333333329, 0.36944444444444446]
--> awetoolbox_calculated_features/_output_uerctest_180class/uerctest-awet_dsift.csv
rows: 1800 -  columns: 12800
Distances calculated ...
[0.155, 0.20555555555555555, 0.24444444444444444, 0.27499999999999997, 0.29777777777777775, 0.31944444444444442, 0.33499999999999996, 0.35555555555555551, 0.37111111111111106, 0.38444444444444437, 0.39388888888888879, 0.41111111111111098, 0.42499999999999988, 0.43555555555555542, 0.44777777777777777]
--> awetoolbox_calculated_features/_output_uerctest_180class/uerctest-awet_gabor.csv
rows: 1800 -  columns:

## TRAIN UERC

## Cosine Distance

In [7]:
from glob import glob 

ufiles = glob("awetoolbox_calculated_features/_output_uerctrain_150class/*.csv")
files = sorted(ufiles)

for i in range(len(files)):
    print("--> {}".format(files[i]))
    res = getMetrics(
        csvfilename = files[i], 
        rank = 15,
        distance_techinque=2) #cosine distance
    print(res)


--> awetoolbox_calculated_features/_output_uerctrain_150class/wputamiuerc-awet_bsif_uerc.csv
rows: 3000 -  columns: 9216
Distances calculated ...
[0.31900000000000001, 0.39733333333333332, 0.44599999999999995, 0.48333333333333328, 0.5139999999999999, 0.54066666666666652, 0.55900000000000005, 0.57500000000000007, 0.58733333333333337, 0.60133333333333339, 0.61366666666666669, 0.629, 0.63966666666666672, 0.65000000000000002, 0.66233333333333322]
--> awetoolbox_calculated_features/_output_uerctrain_150class/wputamiuerc-awet_dsift_uerc.csv
rows: 3000 -  columns: 12800
Distances calculated ...
[0.34200000000000003, 0.41466666666666668, 0.47033333333333338, 0.50533333333333341, 0.53066666666666673, 0.55366666666666675, 0.57833333333333337, 0.59666666666666668, 0.61033333333333339, 0.62666666666666671, 0.63833333333333342, 0.65000000000000013, 0.65833333333333344, 0.66966666666666674, 0.68300000000000005]
--> awetoolbox_calculated_features/_output_uerctrain_150class/wputamiuerc-awet_gabor_uerc

In [10]:
len(files)

8

In [4]:
res = getMetrics(
    csvfilename = "awetoolbox_calculated_features/_output_uerctest_180class/uerctest-awet_lpq.csv", 
    rank = 1,
    distance_techinque=2) #cosine distance
res

rows: 1800 -  columns: 9216
Distances calculated ...


[0.15222222222222223]

In [12]:
res = getMetrics(
    csvfilename = "awetoolbox_calculated_features/_output_uerctest_180class/uerctest-awet_lbp.csv", 
    rank = 1,
    distance_techinque=2) #cosine distance
res

rows: 1800 -  columns: 9971
Distances calculated ...


[0.13166666666666665]

In [13]:
res = getMetrics(
    csvfilename = "awetoolbox_calculated_features/_output_uerctest_180class/uerctest-awet_rilpq.csv", 
    rank = 1,
    distance_techinque=2) #cosine distance
res

rows: 1800 -  columns: 9216
Distances calculated ...


[0.15833333333333333]

In [4]:
res = getMetrics(
    csvfilename = "awetoolbox_calculated_features/_output_uerctest_180class/uerctest-awet_bsif.csv", 
    rank = 1,
    distance_techinque=2) #cosine distance
res

rows: 1800 -  columns: 9216
Distances calculated ...


[0.15333333333333332]

In [5]:
res = getMetrics(S
    csvfilename = "awetoolbox_calculated_features/_output_uerctest_180class/uerctest-awet_poem.csv", 
    rank = 1,
    distance_techinque=2) #cosine distance
res

rows: 1800 -  columns: 11328
Distances calculated ...


[0.18277777777777779]

In [6]:
res = getMetrics(
    csvfilename = "awetoolbox_calculated_features/_output_uerctest_180class/uerctest-awet_dsift.csv", 
    rank = 1,
    distance_techinque=2) #cosine distance
res

rows: 1800 -  columns: 12800
Distances calculated ...


[0.15444444444444444]

In [7]:
res = getMetrics(
    csvfilename = "awetoolbox_calculated_features/_output_uerctest_180class/uerctest-awet_hog.csv", 
    rank = 1,
    distance_techinque=2) #cosine distance
res

rows: 1800 -  columns: 8712
Distances calculated ...


[0.19388888888888889]

In [ ]:
pca_reduce(
    csvfileTrain = "vgg16_calculated_features/vgg16_1stage_wputamiawex_750class.csv", 
    csvfileTest = "vgg16_calculated_features/vgg16_1stage_uerc180class.csv", 
    components = 512)

In [ ]:
lda_reduce(
    csvfileTrain = "vgg16_calculated_features/vgg16_1stage_wputamiawex_750class.csv", 
    csvfileTest = "vgg16_calculated_features/vgg16_1stage_uerc180class.csv", 
    components = 512)

## Test

### Fusion Two Feature Vectors -> LBP + RILPQ

In [ ]:
calculated_features/awe-awet_lbp.csv -> Rank1:0.407
calculated_features/awe-awet_rilpq.csv -> Rank1:0.376
    
calculated_features/awe-awet_hog.csv -> Rank1:0.445
    

#### Method 1
1: Fusion (concatenation)

2: Reduction

In [54]:
csvFiles=["calculated_features/awe-awet_lbp.csv", "calculated_features/awe-awet_rilpq.csv" ]
res = fusionFeatureVectors(csvFiles, 1, "fused_feature_vector/lbp_rilpq_conca.csv")

calculated_features/awe-awet_lbp.csv - rows: 1000 -  columns: 9971
calculated_features/awe-awet_rilpq.csv - rows: 1000 -  columns: 9216
fused_feature_vector/lbp_rilpq_conca.csv - rows: 1000 -  columns: 19187


In [57]:
lda_reduce("fused_feature_vector/lbp_rilpq_conca.csv", 99)

/usr/local/lib/python3.6/dist-packages/sklearn/preprocessing/data.py:323: DataConversionWarning: Data with input dtype int64 were all converted to float64 by MinMaxScaler.
  return self.partial_fit(X, y)
/usr/local/lib/python3.6/dist-packages/sklearn/discriminant_analysis.py:388: UserWarning: Variables are collinear.
  warnings.warn("Variables are collinear.")


New file: fused_feature_vector/lbp_rilpq_conca_redLDA_99.csv


In [58]:
#cosine distance
res = getMetrics("fused_feature_vector/lbp_rilpq_conca_redLDA_99.csv", 1, 2)
res

rows: 1000 -  columns: 99
Distances calculated ...


[0.51300000000000001]

#### Method 2
1: Reduction

2: Fusion (concatenation)

In [86]:
lda_reduce("calculated_features/awe-awet_lbp.csv","calculated_features/cvle-awet_lbp.csv" ,100)
lda_reduce("calculated_features/awe-awet_rilpq.csv","calculated_features/cvle-awet_rilpq.csv" ,100)

/usr/local/lib/python3.6/dist-packages/sklearn/preprocessing/data.py:617: DataConversionWarning: Data with input dtype int64 were all converted to float64 by StandardScaler.
  return self.partial_fit(X, y)
/usr/local/lib/python3.6/dist-packages/sklearn/base.py:462: DataConversionWarning: Data with input dtype int64 were all converted to float64 by StandardScaler.
  return self.fit(X, **fit_params).transform(X)
/usr/local/lib/python3.6/dist-packages/sklearn/discriminant_analysis.py:388: UserWarning: Variables are collinear.
  warnings.warn("Variables are collinear.")


New file: calculated_features/awe-awet_lbp_redLDA_train_99.csv


/usr/local/lib/python3.6/dist-packages/sklearn/preprocessing/data.py:617: DataConversionWarning: Data with input dtype int64 were all converted to float64 by StandardScaler.
  return self.partial_fit(X, y)
/usr/local/lib/python3.6/dist-packages/sklearn/base.py:462: DataConversionWarning: Data with input dtype int64 were all converted to float64 by StandardScaler.
  return self.fit(X, **fit_params).transform(X)


New file: calculated_features/cvle-awet_lbp_redLDA_test_99.csv


/usr/local/lib/python3.6/dist-packages/sklearn/preprocessing/data.py:617: DataConversionWarning: Data with input dtype int64 were all converted to float64 by StandardScaler.
  return self.partial_fit(X, y)
/usr/local/lib/python3.6/dist-packages/sklearn/base.py:462: DataConversionWarning: Data with input dtype int64 were all converted to float64 by StandardScaler.
  return self.fit(X, **fit_params).transform(X)
/usr/local/lib/python3.6/dist-packages/sklearn/discriminant_analysis.py:388: UserWarning: Variables are collinear.
  warnings.warn("Variables are collinear.")


New file: calculated_features/awe-awet_rilpq_redLDA_train_99.csv


/usr/local/lib/python3.6/dist-packages/sklearn/preprocessing/data.py:617: DataConversionWarning: Data with input dtype int64 were all converted to float64 by StandardScaler.
  return self.partial_fit(X, y)
/usr/local/lib/python3.6/dist-packages/sklearn/base.py:462: DataConversionWarning: Data with input dtype int64 were all converted to float64 by StandardScaler.
  return self.fit(X, **fit_params).transform(X)


New file: calculated_features/cvle-awet_rilpq_redLDA_test_99.csv


In [87]:
csvFiles=["calculated_features/cvle-awet_lbp_redLDA_test_99.csv", 
          "calculated_features/cvle-awet_rilpq_redLDA_test_99.csv" ]
res = fusionFeatureVectors(csvFiles, 1, "fused_feature_vector/cvle_lbp_rilpq_redLDA99_conca.csv")

calculated_features/cvle-awet_lbp_redLDA_test_99.csv - rows: 281 -  columns: 99
calculated_features/cvle-awet_rilpq_redLDA_test_99.csv - rows: 281 -  columns: 99
fused_feature_vector/cvle_lbp_rilpq_redLDA99_conca.csv - rows: 281 -  columns: 198


In [88]:
# cosine distance
res = getMetrics("fused_feature_vector/cvle_lbp_rilpq_redLDA99_conca.csv", 1, 2)
res

rows: 281 -  columns: 198
Distances calculated ...


[0.82562277580071175]

In [89]:
lda_reduce("calculated_features/awe-awet_lbp.csv","calculated_features/wput-awet_lbp.csv" ,100)
lda_reduce("calculated_features/awe-awet_rilpq.csv","calculated_features/wput-awet_rilpq.csv" ,100)

/usr/local/lib/python3.6/dist-packages/sklearn/preprocessing/data.py:617: DataConversionWarning: Data with input dtype int64 were all converted to float64 by StandardScaler.
  return self.partial_fit(X, y)
/usr/local/lib/python3.6/dist-packages/sklearn/base.py:462: DataConversionWarning: Data with input dtype int64 were all converted to float64 by StandardScaler.
  return self.fit(X, **fit_params).transform(X)
/usr/local/lib/python3.6/dist-packages/sklearn/discriminant_analysis.py:388: UserWarning: Variables are collinear.
  warnings.warn("Variables are collinear.")


New file: calculated_features/awe-awet_lbp_redLDA_train_99.csv


/usr/local/lib/python3.6/dist-packages/sklearn/preprocessing/data.py:617: DataConversionWarning: Data with input dtype int64 were all converted to float64 by StandardScaler.
  return self.partial_fit(X, y)
/usr/local/lib/python3.6/dist-packages/sklearn/base.py:462: DataConversionWarning: Data with input dtype int64 were all converted to float64 by StandardScaler.
  return self.fit(X, **fit_params).transform(X)


New file: calculated_features/wput-awet_lbp_redLDA_test_99.csv


/usr/local/lib/python3.6/dist-packages/sklearn/preprocessing/data.py:617: DataConversionWarning: Data with input dtype int64 were all converted to float64 by StandardScaler.
  return self.partial_fit(X, y)
/usr/local/lib/python3.6/dist-packages/sklearn/base.py:462: DataConversionWarning: Data with input dtype int64 were all converted to float64 by StandardScaler.
  return self.fit(X, **fit_params).transform(X)
/usr/local/lib/python3.6/dist-packages/sklearn/discriminant_analysis.py:388: UserWarning: Variables are collinear.
  warnings.warn("Variables are collinear.")


New file: calculated_features/awe-awet_rilpq_redLDA_train_99.csv


/usr/local/lib/python3.6/dist-packages/sklearn/preprocessing/data.py:617: DataConversionWarning: Data with input dtype int64 were all converted to float64 by StandardScaler.
  return self.partial_fit(X, y)
/usr/local/lib/python3.6/dist-packages/sklearn/base.py:462: DataConversionWarning: Data with input dtype int64 were all converted to float64 by StandardScaler.
  return self.fit(X, **fit_params).transform(X)


New file: calculated_features/wput-awet_rilpq_redLDA_test_99.csv


In [90]:
csvFiles=["calculated_features/wput-awet_lbp_redLDA_test_99.csv", 
          "calculated_features/wput-awet_rilpq_redLDA_test_99.csv" ]
res = fusionFeatureVectors(csvFiles, 1, "fused_feature_vector/wput_lbp_rilpq_redLDA99_conca.csv")

calculated_features/wput-awet_lbp_redLDA_test_99.csv - rows: 620 -  columns: 99
calculated_features/wput-awet_rilpq_redLDA_test_99.csv - rows: 620 -  columns: 99
fused_feature_vector/wput_lbp_rilpq_redLDA99_conca.csv - rows: 620 -  columns: 198


In [91]:
# cosine distance
res = getMetrics("fused_feature_vector/wput_lbp_rilpq_redLDA99_conca.csv", 1, 2)
res

rows: 620 -  columns: 198
Distances calculated ...


[0.28870967741935483]

In [92]:
lda_reduce("calculated_features/wput-awet_lbp.csv","calculated_features/awe-awet_lbp.csv" ,100)
lda_reduce("calculated_features/wput-awet_rilpq.csv","calculated_features/awe-awet_rilpq.csv" ,100)

/usr/local/lib/python3.6/dist-packages/sklearn/preprocessing/data.py:617: DataConversionWarning: Data with input dtype int64 were all converted to float64 by StandardScaler.
  return self.partial_fit(X, y)
/usr/local/lib/python3.6/dist-packages/sklearn/base.py:462: DataConversionWarning: Data with input dtype int64 were all converted to float64 by StandardScaler.
  return self.fit(X, **fit_params).transform(X)
/usr/local/lib/python3.6/dist-packages/sklearn/discriminant_analysis.py:388: UserWarning: Variables are collinear.
  warnings.warn("Variables are collinear.")


New file: calculated_features/wput-awet_lbp_redLDA_train_100.csv


/usr/local/lib/python3.6/dist-packages/sklearn/preprocessing/data.py:617: DataConversionWarning: Data with input dtype int64 were all converted to float64 by StandardScaler.
  return self.partial_fit(X, y)
/usr/local/lib/python3.6/dist-packages/sklearn/base.py:462: DataConversionWarning: Data with input dtype int64 were all converted to float64 by StandardScaler.
  return self.fit(X, **fit_params).transform(X)


New file: calculated_features/awe-awet_lbp_redLDA_test_100.csv


/usr/local/lib/python3.6/dist-packages/sklearn/preprocessing/data.py:617: DataConversionWarning: Data with input dtype int64 were all converted to float64 by StandardScaler.
  return self.partial_fit(X, y)
/usr/local/lib/python3.6/dist-packages/sklearn/base.py:462: DataConversionWarning: Data with input dtype int64 were all converted to float64 by StandardScaler.
  return self.fit(X, **fit_params).transform(X)
/usr/local/lib/python3.6/dist-packages/sklearn/discriminant_analysis.py:388: UserWarning: Variables are collinear.
  warnings.warn("Variables are collinear.")


New file: calculated_features/wput-awet_rilpq_redLDA_train_100.csv


/usr/local/lib/python3.6/dist-packages/sklearn/preprocessing/data.py:617: DataConversionWarning: Data with input dtype int64 were all converted to float64 by StandardScaler.
  return self.partial_fit(X, y)
/usr/local/lib/python3.6/dist-packages/sklearn/base.py:462: DataConversionWarning: Data with input dtype int64 were all converted to float64 by StandardScaler.
  return self.fit(X, **fit_params).transform(X)


New file: calculated_features/awe-awet_rilpq_redLDA_test_100.csv


In [93]:
csvFiles=["calculated_features/awe-awet_lbp_redLDA_test_100.csv", 
          "calculated_features/awe-awet_rilpq_redLDA_test_100.csv" ]
res = fusionFeatureVectors(csvFiles, 1, "fused_feature_vector/awe_lbp_rilpq_redLDA99_conca.csv")

calculated_features/awe-awet_lbp_redLDA_test_100.csv - rows: 1000 -  columns: 100
calculated_features/awe-awet_rilpq_redLDA_test_100.csv - rows: 1000 -  columns: 100
fused_feature_vector/awe_lbp_rilpq_redLDA99_conca.csv - rows: 1000 -  columns: 200


In [94]:
# cosine distance
res = getMetrics("fused_feature_vector/awe_lbp_rilpq_redLDA99_conca.csv", 1, 2)
res

rows: 1000 -  columns: 200
Distances calculated ...


[0.28199999999999997]

In [59]:
csvFiles=["calculated_features/awe-awet_lbp_redLDA_99.csv", "calculated_features/awe-awet_rilpq_redLDA_99.csv" ]
res = fusionFeatureVectors(csvFiles, 1, "fused_feature_vector/lbp_rilpq_redLDA99_conca.csv")

calculated_features/awe-awet_lbp_redLDA_99.csv - rows: 1000 -  columns: 99
calculated_features/awe-awet_rilpq_redLDA_99.csv - rows: 1000 -  columns: 99
fused_feature_vector/lbp_rilpq_redLDA99_conca.csv - rows: 1000 -  columns: 198


In [60]:
# cosine distance
res = getMetrics("fused_feature_vector/lbp_rilpq_redLDA99_conca.csv", 1, 2)
res

rows: 1000 -  columns: 198
Distances calculated ...


[0.85599999999999998]

In [68]:
csvFiles=["calculated_features/awe-awet_lbp_redLDA_99.csv", 
          "calculated_features/awe-awet_rilpq_redLDA_99.csv", 
          "calculated_features/awe-awet_hog_redLDA_99.csv"]
res = fusionFeatureVectors(csvFiles, 1, "fused_feature_vector/lbp_rilpq_hog_redLDA99_conca.csv")

calculated_features/awe-awet_lbp_redLDA_99.csv - rows: 1000 -  columns: 99
calculated_features/awe-awet_rilpq_redLDA_99.csv - rows: 1000 -  columns: 99
calculated_features/awe-awet_hog_redLDA_99.csv - rows: 1000 -  columns: 99
fused_feature_vector/lbp_rilpq_hog_redLDA99_conca.csv - rows: 1000 -  columns: 297


In [69]:
# cosine distance
res = getMetrics("fused_feature_vector/lbp_rilpq_hog_redLDA99_conca.csv", 1, 2)
res

rows: 1000 -  columns: 297
Distances calculated ...


[0.99299999999999999]

#### Method 2.1
1: Reduction

2: Fusion (sum)

In [62]:
csvFiles=["calculated_features/awe-awet_lbp_redLDA_99.csv", "calculated_features/awe-awet_rilpq_redLDA_99.csv" ]
res = fusionFeatureVectors(csvFiles, 2, "fused_feature_vector/lbp_rilpq_redLDA99_sum.csv")

calculated_features/awe-awet_lbp_redLDA_99.csv - rows: 1000 -  columns: 99
calculated_features/awe-awet_rilpq_redLDA_99.csv - rows: 1000 -  columns: 99
fused_feature_vector/lbp_rilpq_redLDA99_sum.csv - rows: 1000 -  columns: 99


In [64]:
# cosine distance
res = getMetrics("fused_feature_vector/lbp_rilpq_redLDA99_sum.csv", 1, 2)
res

rows: 1000 -  columns: 99
Distances calculated ...


[0.72199999999999998]

#### Method 2.2
1: Reduction

2: Fusion (max)

In [66]:
csvFiles=["calculated_features/awe-awet_lbp_redLDA_99.csv", "calculated_features/awe-awet_rilpq_redLDA_99.csv" ]
res = fusionFeatureVectors(csvFiles, 3, "fused_feature_vector/lbp_rilpq_redLDA99_max.csv")

calculated_features/awe-awet_lbp_redLDA_99.csv - rows: 1000 -  columns: 99
calculated_features/awe-awet_rilpq_redLDA_99.csv - rows: 1000 -  columns: 99
fused_feature_vector/lbp_rilpq_redLDA99_max.csv - rows: 1000 -  columns: 99


In [67]:
# cosine distance
res = getMetrics("fused_feature_vector/lbp_rilpq_redLDA99_max.csv", 1, 2)
res

rows: 1000 -  columns: 99
Distances calculated ...


[0.57299999999999995]

In [36]:
csvFiles=["calculated_features/awe-awet_lbp_redLDA_99.csv", "calculated_features/awe-awet_rilpq_redLDA_99.csv" ]
res = fusionFeatureVectors(csvFiles, 2, "fused_feature_vector/lbp_rilpq_redLDA99_sum.csv")

calculated_features/awe-awet_lbp_redLDA_99.csv - rows: 1000 -  columns: 99
calculated_features/awe-awet_rilpq_redLDA_99.csv - rows: 1000 -  columns: 99
fused_feature_vector/lbp_rilpq_redLDA99_sum.csv - rows: 1000 -  columns: 99


In [42]:
csvFiles=["calculated_features/awe-awet_lbp_redLDA_99.csv", "calculated_features/awe-awet_rilpq_redLDA_99.csv" ]
res = fusionFeatureVectors(csvFiles, 3, "fused_feature_vector/lbp_rilpq_redLDA99_max.csv")

calculated_features/awe-awet_lbp_redLDA_99.csv - rows: 1000 -  columns: 99
calculated_features/awe-awet_rilpq_redLDA_99.csv - rows: 1000 -  columns: 99
fused_feature_vector/lbp_rilpq_redLDA99_max.csv - rows: 1000 -  columns: 99


In [47]:
pca_reduce("fused_feature_vector/lbp_rilpq.csv", 1000)

/usr/local/lib/python3.6/dist-packages/sklearn/preprocessing/data.py:323: DataConversionWarning: Data with input dtype int64 were all converted to float64 by MinMaxScaler.
  return self.partial_fit(X, y)


## Distances

In [45]:
res = getMetrics("fused_feature_vector/lbp_rilpq_redLDA_99.csv", 1, 1)
res

rows: 1000 -  columns: 99


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:6: RuntimeWarning: invalid value encountered in sqrt
  


Distances calculated ...


[0.0089999999999999993]

In [48]:
res = getMetrics("fused_feature_vector/lbp_rilpq_redLDA_99.csv", 1, 3)
res

rows: 1000 -  columns: 99
Distances calculated ...


[0.33500000000000002]

In [ ]:
from glob import glob

files = glob('calculated_features/*.csv')
for f in files:
    pca_reduce(f,1000)
    lda_reduce(f,100)